
# Профилирование кода: `timeit`, `cProfile`, `line_profiler`



## Что такое профилирование

**Профилирование** — это процесс анализа производительности программы.  
Оно помогает определить, какие части кода требуют оптимизации.

**Основные задачи профилирования:**
- Измерение времени выполнения кода.
- Определение узких мест (bottlenecks).
- Сравнение эффективности разных реализаций алгоритмов.



## 1. Модуль `timeit`

`timeit` — стандартный инструмент Python для измерения времени выполнения небольших фрагментов кода.

Он автоматически выполняет код несколько раз и усредняет результаты, чтобы исключить случайные колебания.




In [2]:

import timeit

setup_code = "pass"
code_1 = "[i for i in range(1000)]"
code_2 = "list(range(1000))"

time1 = timeit.timeit(stmt=code_1, setup=setup_code, number=10000)
time2 = timeit.timeit(stmt=code_2, setup=setup_code, number=10000)

print(f"List comprehension: {time1:.5f} сек")
print(f"list(range()): {time2:.5f} сек")


List comprehension: 0.13731 сек
list(range()): 0.06406 сек



## 1. Параметр setup

    Нужен для частей кода, которые компилируются единоразово:

    1. Импорта модулей и функций

    2. Создания тестовых данных

    3. Инициализации переменных




In [5]:

import timeit


time_bad = timeit.timeit(
    stmt='import math; math.sqrt(144)',
    number=10000
)


time_good = timeit.timeit(
    setup='import math',
    stmt='math.sqrt(144)',
    number=10000
)

print(f"Без setup: {time_bad:.6f} сек")
print(f"С setup:   {time_good:.6f} сек")
print(f"Разница:   {time_bad - time_good:.6f}")


Без setup: 0.002253 сек
С setup:   0.000720 сек
Разница:   0.001534



### Основные функции и классы `timeit`

| Команда | Назначение |
|----------|------------|
| `timeit.timeit(stmt, setup, number)` | Выполняет одно измерение времени выполнения кода. |
| `timeit.repeat(stmt, setup, repeat, number)` | Проводит несколько серий измерений, возвращает список времен. |
| `timeit.Timer` | Класс для продвинутого управления измерениями и многократного запуска. |
| `timeit.default_timer()` | Возвращает наиболее точный таймер для данной платформы. (учитывает версию питона, операционной системы) |
| `Timer.autorange()` | Автоматически подбирает количество итераций для стабильного результата. |


In [4]:
import timeit

times = timeit.repeat("sum(range(1000))", repeat=5, number=1000)
print("Результаты повторов:", times)

timer = timeit.Timer("sum(range(100))") # дальше с ним возможно использовать так и timeit repeat и другие функции
print("Автоматический подбор количества итераций:", timer.autorange())

start = timeit.default_timer()
sum(range(1000000))
end = timeit.default_timer()
print(f"Время (default_timer): {end - start:.6f} сек")


Результаты повторов: [0.01891169999998965, 0.02390739999998459, 0.01810879999993631, 0.01692460000003848, 0.0177513999999519]
Автоматический подбор количества итераций: (500000, 0.44537350000007336)
Время (default_timer): 0.019340 сек



### Недостатки `timeit`
- Только общее время выполнения.
- Не показывает, где именно код замедляется. Что делает неудобным для больших програм



## 2. Модуль `cProfile`

`cProfile` — встроенный профайлер Python для анализа времени выполнения функций и количества их вызовов.



### Основные функции и классы `cProfile`

| Команда | Назначение |
|----------|------------|
| `cProfile.run(cmd)` | Быстрое профилирование одной команды (строки кода). |
| `cProfile.runctx(cmd, globals, locals)` | Профилирование с передачей контекста (переменных окружения). |
| `cProfile.Profile()` | Класс для ручного управления профилированием. |
| `enable()` / `disable()` | Включение/выключение профилирования вручную. |


In [5]:

import cProfile, pstats

def compute():
    s = 0
    for i in range(500):
        for j in range(500):
            s += i * j
    return s

#метод 1
cProfile.run('compute()')

#метод 2
context = {"compute": compute}
cProfile.runctx('compute()', globals(), context)

#метод 3
prof = cProfile.Profile() # Создание профилировщика
prof.enable() # включаем ручной контроль
compute() 
prof.disable() # выключаем ручной контроль

stats = pstats.Stats(prof)
stats.sort_stats(pstats.SortKey.TIME).print_stats(5)


         4 function calls in 0.008 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.008    0.008    0.008    0.008 1077826651.py:3(compute)
        1    0.000    0.000    0.008    0.008 <string>:1(<module>)
        1    0.000    0.000    0.008    0.008 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


         89 function calls in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.001    0.001 1077826651.py:3(compute)
        1    0.000    0.000    0.001    0.001 <string>:1(<module>)
        6    0.000    0.000    0.000    0.000 enum.py:1156(__new__)
       18    0.000    0.000    0.000    0.000 enum.py:1589(_get_value)
        6    0.000    0.000    0.000    0.000 enum.py:1596(__or__)
        6    0.000    0.000    0.000    0.000 enum


### Недостатки `cProfile`
- Показатели усреднены по функциям, а не по строкам.
- Неудобно искать конкретные медленные строки.



## 3. Модуль `line_profiler`

`line_profiler` — инструмент для **построчного анализа** времени выполнения функций.

Устанавливается отдельно:
```
pip install line_profiler
```



### Основные элементы `line_profiler`

| Команда | Назначение |
|----------|------------|
| `@profile` | Декоратор для профилирования через утилиту `kernprof`. |
| `LineProfiler()` | Основной класс для ручного профилирования. |
| `add_function(func)` | Добавление функций для анализа. |
| `enable_by_count()` / `disable_by_count()` | Управление активностью профайлера. |
| `print_stats()` | Вывод статистики в консоль. |
| `get_stats()` | Получение «сырых» данных о времени выполнения. |


In [4]:

from line_profiler import LineProfiler

def slow_function():
    total = 0
    for i in range(1000):
        for j in range(1000):
            total += (i * j) % 7
    return total

prof = LineProfiler()
prof.add_function(slow_function)
prof.enable_by_count()
slow_function()
prof.disable_by_count()
prof.print_stats()


Timer unit: 1e-07 s

Total time: 0.560883 s
File: C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10952\1192547859.py
Function: slow_function at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def slow_function():
     4         1          8.0      8.0      0.0      total = 0
     5      1001       2825.0      2.8      0.1      for i in range(1000):
     6   1001000    2781075.0      2.8     49.6          for j in range(1000):
     7   1000000    2824905.0      2.8     50.4              total += (i * j) % 7
     8         1         18.0     18.0      0.0      return total




### Использование декоратора `@profile`

```python
@profile
def slow_function():
    total = 0
    for i in range(1000):
        for j in range(1000):
            total += (i * j) % 7
    return total
```

Запуск через консоль:
```
kernprof -l -v script.py
```



### Недостатки `line_profiler`
- Не входит в стандартную библиотеку Python.
- Требует отдельного запуска или настройки расширений.
- Медленнее других методов из-за построчного анализа.



## 4. Сравнение инструментов профилирования

| Инструмент | Уровень детализации | Простота использования | Встроен в Python | Основное применение |
|-------------|----------------------|-------------------------|------------------|----------------------|
| `timeit` | Измеряет общее время выполнения блока кода | Очень прост | ✅ Да | Сравнение небольших фрагментов |
| `cProfile` | Анализ по функциям | Средний | ✅ Да | Профилирование больших программ |
| `line_profiler` | Построчный анализ | Чуть выше среднего | ❌ Нет | Оптимизация конкретных функций |



## Итоги

- **`timeit`** — лучший выбор для коротких участков кода и сравнения алгоритмов.  
- **`cProfile`** — оптимален для комплексного анализа больших программ.  
- **`line_profiler`** — незаменим при необходимости найти медленные строки в конкретных функциях.
